# Multiple Linear Regression Model #

### Time Series Regression Explaining Stock's Returns ###

In [145]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Visuals
import seaborn as sns
import matplotlib.pyplot as plt

# Statistics
import statsmodels.api as sm 
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import t

# Pretty Notation
from IPython.display import display, Math

In [146]:
# The Matrix Form of the OLS Linear Regression Model

display(Math(r"Y = X\beta+\varepsilon"))

In [147]:
# Data S&P500

sp500 = pd.read_csv(rf"..\additional_data\sp500.csv")
sp500.set_index('Date', inplace = True)
sp500.index = pd.to_datetime(sp500.index)
sp500.dropna(inplace = True)

# Cut the sample
sp500 = sp500.loc['2015-01-01':]

sp500

In [148]:
# Call the stock's data

data_stock = pd.read_csv(rf"..\stocks\AMZN.csv")
data_stock.set_index('Date', inplace = True)
data_stock.index = pd.to_datetime(data_stock.index)
data_stock.dropna(inplace = True)

# Cut the sample
data_stock = data_stock.loc['2015-01-01':]

data_stock

In [149]:
# Create the Data we will need

data_regression = pd.DataFrame(index=data_stock.index)

# Now safely add new columns
data_regression['returns'] = data_stock['Adjusted_close'].pct_change(1).mul(100)
data_regression['mkt_returns'] = sp500.pct_change(1).mul(100)
data_regression['log_mktcap'] = np.log(data_stock['Company Market Cap'])
data_regression['annualized_volat'] = data_regression['returns'].rolling(window=252).std() * np.sqrt(252)
data_regression['ptb'] = data_stock['Price_to_Book']

# Drop nans
data_regression.dropna(inplace = True)

data_regression

In [150]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8), dpi=600, sharex=True)  # 2 filas, 2 columnas

# First Graph
data_regression['returns'].cumsum().plot(ax=axs[0, 0], color='blue', label='Stock Price')
axs[0, 0].set_title('Price')
axs[0, 0].legend()

# Second Graph
data_regression['log_mktcap'].plot(ax=axs[0, 1], color='orange', label='Log Market Cap')
axs[0, 1].set_title('Log Market Cap')
axs[0, 1].legend()

# Third Graph
data_regression['annualized_volat'].plot(ax=axs[1, 0], color='green', label='Anualized Rolling Volatility')
axs[1, 0].set_title('Volatility')
axs[1, 0].legend()

# Fourth
data_regression['ptb'].plot(ax=axs[1, 1], color='red', label='Price to Book')
axs[1, 1].set_title('Price to Book')
axs[1, 1].legend()

# Show
plt.tight_layout()
plt.show()

In [151]:
# Correlation Matrix

data_regression.corr()

In [152]:

# Setup subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 8), dpi=600)

# Variables to plot against 'returns'
variables = ['mkt_returns', 'log_mktcap', 'annualized_volat', 'ptb']
titles = ['Market Returns', 'Log Market Cap', 'Annualized Volatility', 'Price to Book']
colors = ['orange', 'green', 'red', 'purple']

# Create scatter plots
for i, (var, title, color) in enumerate(zip(variables, titles, colors)):
    row, col = divmod(i, 2)
    axs[row, col].scatter(data_regression[var], data_regression['returns'], alpha=0.5, color=color)
    axs[row, col].set_title(f'Returns vs {title}')
    axs[row, col].set_xlabel(title)
    axs[row, col].set_ylabel('Returns')

# Layout and show
plt.tight_layout()
plt.show()

In [153]:
filtered_data = data_regression[(np.abs(data_regression - data_regression.mean()) <= 3 * data_regression.std())]

filtered_data.dropna(inplace = True)

filtered_data

In [154]:
# The Correlation Matrix

filtered_data.corr()

In [155]:
# Create the Y Vector

Y_Vector = filtered_data["returns"]

Y_Vector

In [156]:
# Create the Information Matrix
# Remember to add a constant
filtered_data['squared_volat'] = filtered_data['annualized_volat'].mul(filtered_data['annualized_volat'])

Information_Matrix = filtered_data.copy().drop('returns', axis=1)
Information_Matrix = sm.add_constant(Information_Matrix)

Information_Matrix

In [157]:
# For estimating the coefficients with the OLS we need to assess the non-perfect collinearity condition with the matrix rank

Rank = np.linalg.matrix_rank(Information_Matrix)

# Since we are estimating four betas, the rank must be the number of columns in the Information Matrix

if Rank == 6:
    print(f"Matrix Rank is {Rank}; there is no evidence of Perfect Multicollinearity.")
else:
    print(f"Matrix Rank is {Rank}; there is evidence of Perfect Multicollinearity between two or more variables.")

In [158]:
# The beta calculation in the Matrix Form

display(Math(r"\beta=(X^⊤X)^{-1}(X^⊤Y)"))

# The vector of betas is a kx1 vector containing the coefficients of the regression model, where k is the number of parameters to estimate

In [159]:
# Transpose of the Information Matrix

Information_Matrix_T = Information_Matrix.transpose()

Information_Matrix_T

In [160]:
# Now we have to calculate the (X^⊤X)^{-1}, first the square of the Information Matrix

Information_Matrix_Square = Information_Matrix_T.dot(Information_Matrix)

# We could also use the command Information_Matrix_T @ Information_Matrix

Information_Matrix_Square

In [161]:
# The reason we needed to know the rank of the information matrix is because, only full rank matrix can be invertible
# Then we can calculate the Matrix Determinant, if it is different from zero, we can calculate the OLS coefficients

Information_Matrix_Square_Determinant = np.linalg.det(Information_Matrix_Square)

if Information_Matrix_Square_Determinant != 0:
    print(f"The Determinant of the Squared Information Matrix is {Information_Matrix_Square_Determinant} and different from zero")
else:
    print("Matrix NOT invertible")

In [162]:
# Now we have to obtain the Inverse Matrix

X_Variance_Matrix_Inverse = np.linalg.inv(Information_Matrix_Square)

X_Variance_Matrix_Inverse

In [163]:
# Now we have to obtain (X^⊤Y)

Y_Covariance_X = Information_Matrix_T.dot(Y_Vector)

Y_Covariance_X

In [164]:
# Now we can calculate the Betas

Beta = X_Variance_Matrix_Inverse.dot(Y_Covariance_X)

Beta_DF = pd.DataFrame(Beta, index = Information_Matrix.columns)
Beta_DF

In [165]:
# Now we can obtain the fitted values

Y_Hat = Information_Matrix.dot(Beta)

Y_Hat

In [166]:
fig, ax1 = plt.subplots(dpi = 600)

filtered_data['expected_returns'] = Y_Hat

filtered_data['returns'].cumsum().plot(label = 'Observed')
filtered_data['expected_returns'].cumsum().plot(label = 'Fitted')
plt.legend()
plt.title('Observed vs Fitted')

plt.show()

In [167]:
# We can obtain the Hat Matrix which is a matrix used to transform the real values into the fitted values

Some_Matrix = Information_Matrix.dot(X_Variance_Matrix_Inverse)
Hat_Matrix = Some_Matrix.to_numpy() @ Information_Matrix_T.to_numpy()

Hat_Matrix

In [168]:
print(Information_Matrix.shape)
print(X_Variance_Matrix_Inverse.shape)
print(Some_Matrix.shape)
print(Information_Matrix_T.shape)

In [169]:
# Let us check if this is true
Y_Hat_2 = Hat_Matrix.dot(Y_Vector)

Y_Hat_2

In [170]:
# Hat Matrix is Symmetric and Idempotent

Hat_Matrix_Square = (Hat_Matrix.transpose()).dot(Hat_Matrix)

if Hat_Matrix.all() == Hat_Matrix_Square.all():
    print("It is indeed idempotent")
else:
    print("Wrong!")

In [171]:
# We can calculate the residuals using the Hat Matrix

Identity_Matrix = np.identity(len(Y_Vector))
Residuals_Vector = (Identity_Matrix - Hat_Matrix).dot(Y_Vector)

print(f"The Residuals Mean is: {Residuals_Vector.mean().round(3)}")
print(f"The Residuals Variance is: {Residuals_Vector.var()}")

In [172]:
# Residual Returns
filtered_data['residuals'] = Residuals_Vector

filtered_data

In [173]:
# Plot
fig, ax1 = plt.subplots(dpi = 600)
filtered_data['residuals'].cumsum().plot()
plt.title('Residual Returns Time Series')
plt.show()

In [174]:
# The OLS Assumptions establish that the covariates and the residuals must be uncorrelated

Intercorrelation_Vector = Information_Matrix_T.dot(Residuals_Vector)

print(Intercorrelation_Vector.round(5))

In [175]:
# We can use this to calculate the Bias of the Beta coefficients

display(Math(r"S=(X^⊤X)^{-1}(X^⊤\varepsilon)"))

Bias = X_Variance_Matrix_Inverse.dot(Intercorrelation_Vector)

print("""
Biases are very close to zero.
""")
print(Bias)

In [176]:
# Now we want to calculate the sum of squares

display(Math(r"RSS=\varepsilon^⊤\varepsilon=Y^⊤Y-\beta^⊤X^⊤Y"))
display(Math(r"ESS=\beta^⊤X^⊤Y-\bar{Y}^2"))
display(Math(r"TSS=Y^⊤Y-\bar{Y}^2"))

In [177]:
RSS = (Residuals_Vector.transpose()).dot(Residuals_Vector)
ESS = (Beta.transpose()).dot(Y_Covariance_X) - (sum(Y_Vector)**2)/len(Y_Vector)
TSS = (Y_Vector.transpose()).dot(Y_Vector) - (sum(Y_Vector)**2)/len(Y_Vector)

print(f"The Residuals Sum of Squares is: {RSS}")
print(f"The Estimation Sum of Squares is: {ESS}")
print(f"The Total Sum of Squares is: {TSS}")

In [178]:
# We can calculate the R-Squared Coefficient

R_Squared = ESS/TSS

print(f"The R-Squared Coefficient is: {R_Squared}")

In [179]:
# Now calculate the Residuals Variance with n - k degrees of freedom (adjusted to the sample)

Residuals_Variance = RSS/(len(Y_Vector) - Hat_Matrix.trace())

print(f"The Residuals Variance is: {Residuals_Variance}")

In [180]:
# We use the Residuals Variance to calculate the covariances between all the beta coefficients

display(Math(r"C=\frac{\varepsilon^⊤\varepsilon}{n-k}(X^⊤X)^{-1}"))

In [181]:
# The Diagonal of the Covariance Matrix contains the standard errors of the beta coefficients

Covariance_Matrix = (Residuals_Variance)*X_Variance_Matrix_Inverse

Covariance_Matrix

In [182]:
# Take the squared-root

Beta_Standards_Errors = np.sqrt(Covariance_Matrix.diagonal())

Beta_Standards_Errors

In [183]:
# Calculate the T-Values

T_Values = Beta/Beta_Standards_Errors

T_Values

In [184]:
# How much degrees of freedom we have?

df = len(Y_Vector) - Hat_Matrix.trace()

print(f"We have {df.round()} degrees of freedom.")

In [185]:
# The Hypothesis Testing implies to reject the null hypothesis if the t-values are higher than the critic t-value
# For 293 degrees of freedom the critic t-value approaches to 1.96
# Then we can calculate the upper and lower limits

Beta_Lower_Limit = Beta - 1.96*Beta_Standards_Errors
Beta_Upper_Limit = Beta + 1.96*Beta_Standards_Errors

print(Beta_Lower_Limit)
print(Beta_Upper_Limit)

In [186]:
# We can build a dataframe that contains all the information

Proof_DF = pd.DataFrame(
    {
     "T_Values": T_Values, 
     "Beta_Inferior_Limit": Beta_Lower_Limit, 
     "Beta_Superior_Limit": Beta_Upper_Limit
     }
    )

print(Proof_DF)

In [187]:
# Let us obtain the p-values, if these are less than 0.05, we reject the null hypothesis confirming statistically significance

Proof_DF["p-values"] = 2*(t.sf(
    abs(Proof_DF.T_Values), 
    len(Y_Vector) - Hat_Matrix.trace()
    ).round(3)
    )

print(Proof_DF)

In [188]:
"""
The R-Squared is not always the most precise staitstic in a multilinear model
The Adjusted R-Squared penalizes the existence of more variables in our model:
    
    Adjusted R Squared = 1 – [((1 – R2) * (n – 1)) / (n – k)]
    
"""

R_Squared_Adjusted = (1 - ((1-R_Squared)*(len(Y_Vector) - 1)/(len(Y_Vector) - Hat_Matrix.trace())))

print(f"The Adjusted R-Squared is: {R_Squared_Adjusted.round(5)}")

In [189]:
"""
The F Statistic  help us to prove Joint Significance.
This means taht we are checking if our models as a whole can explain the Y

The F statistic is distributed in a F distribution with n - k and
k - 1 degrees of freedom.
"""

display(Math(r"F=\frac{ESS/k-1}{RSS/n-k}"))


In [190]:
# Calculate the F Stat

F_Stat = (ESS/(Hat_Matrix.trace() - 1)) / (RSS/(len(Y_Vector) - Hat_Matrix.trace()))

print(f"The F-Statistic is: {F_Stat}")

In [191]:
#Model specification
model = sm.OLS(
    Y_Vector, 
    sm.add_constant(Information_Matrix)
    )   
     
#the results of the model
results = model.fit() 
    
#The Parameters
Beta2 = results.params  

#here we check the summary
print(results.summary())       

In [192]:
Beta

In [193]:
# Calculate the VIF

X = Information_Matrix

vif_data = pd.DataFrame()
vif_data['vars'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data

In [194]:
r_squared_df = pd.DataFrame()
r_squared_df['vars'] = X.columns

r_squared_df['r_squared'] = 1 - (1 / vif_data['VIF'])

r_squared_df

In [195]:
#Model specification
model = sm.OLS(
    Y_Vector, 
    sm.add_constant(X.drop(['annualized_volat', 'squared_volat'], axis=1))
    )   
     
#the results of the model
results = model.fit() 

#here we check the summary
print(results.summary())   

In [196]:
# Calculate the VIF

X = Information_Matrix.drop(['annualized_volat', 'squared_volat'], axis=1)

vif_data = pd.DataFrame()
vif_data['vars'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data